# Find example annotations

In [1]:
import sys
if('..' not in sys.path):
    sys.path.append('..')
from lqam.util.file_utils import cached_path
from lqam.methods.dataset import URL_DATA_TEST, URL_TEST_LABEL_CATEGORIES
import json
import pandas as pd
test_data = pd.DataFrame(json.load(open(cached_path(URL_DATA_TEST))))
test_data_categories = pd.read_csv(cached_path(URL_TEST_LABEL_CATEGORIES), sep='\t', index_col=False)

In [2]:
## join 
test_data = pd.merge(test_data, test_data_categories.loc[:, ['video_id', 'category']], on='video_id')

In [3]:
## flatten, normalize answers
from lqam.core.metrics import normalize_answer
test_data = test_data.assign(**{
    'flat_answers' : test_data.loc[:, 'additional_answers'].apply(lambda x: [normalize_answer(z) for y in x for z in y])
})

In [4]:
test_data.drop_duplicates('category').loc[:, ['category', 'label']]

,category,label
0,0,the art
3,4,himself
4,-1,a metal
5,2,the field
8,7,some poppy music
12,1,A young girl
30,6,a routine
48,3,a small crowd
75,9,an old ad
91,8,accident


In [5]:
category_map = {
    0 : 'thing',
    1 : 'person',
    2 : 'location',
    3 : 'phrase',
    4 : 'pronoun',
    5 : 'event',
    6 : 'action',
    7 : 'audio',
    8 : 'abstract',
    9 : 'other',
    -1 : 'NA',
}
test_data = test_data.assign(**{
    'category_name' : test_data.loc[:, 'category'].apply(category_map.get)
})
print(test_data.loc[:, 'category_name'].value_counts())

thing       394
person      316
pronoun      59
location     54
phrase       44
action       38
audio        24
abstract     21
NA           19
other        15
event        10
Name: category_name, dtype: int64


What are some example captions for person, thing, and location?

In [8]:
pd.set_option('display.max_colwidth', 250)
sample_size = 5
sample_categories = ['thing', 'person', 'location']
for sample_category_i in sample_categories:
    print(f'**** category = {sample_category_i} ****')
    data_i = test_data[test_data.loc[:, 'category_name']==sample_category_i] 
    display(data_i.iloc[:sample_size, :].loc[:, ['category_name', 'video_start_time', 'video_end_time', 'masked_caption', 'label', 'flat_answers']])

**** category = thing ****


,category_name,video_start_time,video_end_time,masked_caption,label,flat_answers
0,thing,37,47,A man in an outside field is demonstrating _____ of throwing a javelin.,the art,"[technique, skill, best way, technique, method, practice, training, instructions, way, technique, art, his style, correct way, regular way, finer points, skill, technique, technique, way, form, position, technique, method]"
1,thing,16,26,Two people intentionally step on and crush juicy grapes on _____.,a sidewalk,"[ground, dirt, sidewalk, ground, soil, ground, pavement, floor, floor, video, ground, earth, ground, side of road, ground, pavement, pavement, sidewalk, ground, dirt]"
2,thing,76,86,A man is sitting down in a chair simulating using _____ while he is rapping.,a scratching table,"[mixer, dj equipment, microphone, beatboxing, percussion, multiple sounds, beat boxing, microphone, mic, turn table, instrument, mike, microphone, turntables, dj equipment, microphone, speaker, beat, music]"
6,thing,0,10,"A man is looking straight into the camer and cracking his neck, the camera then pans to _____.",his messy computer desk,"[his computer, his messy desk, desk, computer desk, work station, two tiered desk, pull out desk, keyboard, computer, desk, keyboard, computer, some medication, keyboard, table, computer, trash, computer, keyboard, desk, computer keyboard, his co..."
10,thing,13,23,"A man in a suit, holding _____, is outside jumping insie a blowup bouncy device as people are walking by watching him.",a briefcase,"[briefcase, case, briefcase, black bag, black briefcase, bag, briefcase, briefcase, bag, briefcase, his workback, black briefcase, duffle bag, briefcase, bag, suitcase, luggage, bag, briefcase]"


**** category = person ****


,category_name,video_start_time,video_end_time,masked_caption,label,flat_answers
12,person,73,83,_____ in a stable brushing a white horse's hair.,A young girl,"[women, person, girl wearing helmet, young girl, child, girl, kid, girl, child, adolescent, female, lady wearing jockey cap, woman, horse groomer, rider, person, lady, girl, child, girl, young lady]"
17,person,54,64,A group of _____ together outdoors in the snow playing with snowmen.,children,"[kids, girls, children, humans, children, kids, girls, kids, sisters, children, kids, children, kids, children, little ones, tykes, small fries, kids, sisters, children, kids, siblings, sisters, adolescents, kids, children]"
20,person,0,10,_____ at the front of a church while singing and the church sang along.,A choir,"[man, person, woman, lady, person standing, man standing, praising, jumping, woman, woman, individual]"
22,person,1,11,"A group of people are riding donkeys, some people are riding and _____ are just standing by them.",others,"[others, others, others, others, others, other people, some people, others, others, some people]"
23,person,58,68,_____ is sitting down in a crowded marketplace playing an instrument.,A man,"[man, person, man, person, man, musician, person, white guy, man, gentleman, man, musician, guy, man, hobo, man with curly hair, man in white sleeveless shirt, man, man, guy]"


**** category = location ****


,category_name,video_start_time,video_end_time,masked_caption,label,flat_answers
5,location,199,209,Two kids are egg hunting in _____ while talking and running.,the field,"[yard, back yard, grass, yard, lawn, field, park, field, grass, park, garden, field, grass, meadow, nature, forest, yard, backyard, trees, yard, backyard, grass, outdoors, backyard, grassy area, open area, park like area, peaceful setting, backya..."
51,location,15,25,A group of people at a soccer game in _____.,the stands,"[stadium, field, stadium, stands, soccer field, bleachers, stands, crowd, circle, person, stadium, stadium, soccer stadium, stadium, huddle, stadium, outdoor stadium]"
61,location,40,50,A group of people in _____ are standing around singing and clapping.,church,"[church, church, stadium, church, assembly hall, church, chapel, church, worship, church, large room, church, house of worship, church, temple, sanctuary, church, worship]"
74,location,0,10,A person slowly abseils down _____ of a building.,the side,"[side, wall, outside, outside, side, front, side, wall, side, back, wall, side, wall, side, wall, side, face, side, wall, height, outside, side, wall]"
79,location,162,172,Two young men are seated at a table and one places a doughnut on _____ of another and holds them in both hands.,top,"[top, top, plate, box, top, bottom, hands, box, top, frosting, top, topside, top, bottom, top, icing]"


In [13]:
sample_video_ids = [
    '5ZxOxGrEB0E', # thing
    '979dp52LgcI', # person
    'z3y5nZiyfgs', # location
]
for video_id_i in sample_video_ids:
    data_i = test_data[test_data.loc[:, 'video_id']==video_id_i]
    # replace answers with set
    data_i = data_i.assign(**{'flat_answers' : data_i.loc[:, 'flat_answers'].apply(lambda x: list(sorted(set(x))))})
    display(data_i.loc[:, ['video_id', 'video_start_time', 'video_end_time', 'masked_caption', 'label', 'flat_answers', 'category_name']])

,video_id,video_start_time,video_end_time,masked_caption,label,flat_answers,category_name
54,5ZxOxGrEB0E,9,19,Two children throw _____ at each other as a video is captured in slow motion.,water balloons,"[balloons, balloons filled with water, balloons of water, pink balloon, pink water balloon, things, water, water balloons, water filled balloons, waterfilled balloons]",thing


,video_id,video_start_time,video_end_time,masked_caption,label,flat_answers,category_name
150,979dp52LgcI,5,15,_____ sits at a drum set and practices playing the drums.,A young girl,"[child, drummer, future drummer, girl, kid, little girl, little kid, musician, small child, young girl]",person


,video_id,video_start_time,video_end_time,masked_caption,label,flat_answers,category_name
174,z3y5nZiyfgs,69,79,A man is walking around _____ hugging people on National Hug Day.,a food court,"[attempting, building, cafeteria, courtyard, food court, holding poster, mall, mall food court, room, shopping center, small mall, store]",location
